<a href="https://colab.research.google.com/github/AriSalassa/procesamiento_lenguaje_natural/blob/main/clase_3/ejercicios/ejercicio_clase_3_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Custom embedddings con Gensim



### Objetivo
El objetivo es utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto. Se utilizará canciones de bandas para generar los embeddings, es decir, que los vectores tendrán la forma en función de como esa banda haya utilizado las palabras en sus canciones.

In [1]:
!pip install gensim==3.8.3

     |████████████████████████████████| 24.2 MB 1.5 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec

### Datos
Utilizaremos como dataset canciones de bandas de habla inglés.

In [3]:
# Descargar la carpeta de dataset
import os
import platform
if os.access('./songs_dataset', os.F_OK) is False:
    if os.access('songs_dataset.zip', os.F_OK) is False:
        if platform.system() == 'Windows':
            !curl https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip -o songs_dataset.zip
        else:
            !wget songs_dataset.zip https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/datasets/songs_dataset.zip
    !unzip -q songs_dataset.zip   
else:
    print("El dataset ya se encuentra descargado")

--2022-04-06 21:45:28--  http://songs_dataset.zip/
Resolving songs_dataset.zip (songs_dataset.zip)... failed: Name or service not known.
wget: unable to resolve host address ‘songs_dataset.zip’
--2022-04-06 21:45:28--  https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/datasets/songs_dataset.zip
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip [following]
--2022-04-06 21:45:28--  https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.

In [4]:
# Posibles bandas
list_dir = os.listdir("./songs_dataset/")
print(len(list_dir))
list_dir

49


['Lil_Wayne.txt',
 'kanye.txt',
 'notorious_big.txt',
 'blink-182.txt',
 'nicki-minaj.txt',
 'nursery_rhymes.txt',
 'bieber.txt',
 'patti-smith.txt',
 'nickelback.txt',
 'bruce-springsteen.txt',
 'lin-manuel-miranda.txt',
 'al-green.txt',
 'michael-jackson.txt',
 'adele.txt',
 'paul-simon.txt',
 'prince.txt',
 'drake.txt',
 'disney.txt',
 'jimi-hendrix.txt',
 'britney-spears.txt',
 'kanye-west.txt',
 'radiohead.txt',
 'missy-elliott.txt',
 'alicia-keys.txt',
 'dolly-parton.txt',
 'johnny-cash.txt',
 'cake.txt',
 'bob-dylan.txt',
 'lorde.txt',
 'beatles.txt',
 'ludacris.txt',
 'eminem.txt',
 'leonard-cohen.txt',
 'joni-mitchell.txt',
 'dj-khaled.txt',
 'janisjoplin.txt',
 'dr-seuss.txt',
 'lil-wayne.txt',
 'Kanye_West.txt',
 'dickinson.txt',
 'bob-marley.txt',
 'lady-gaga.txt',
 'r-kelly.txt',
 'notorious-big.txt',
 'nirvana.txt',
 'amy-winehouse.txt',
 'bruno-mars.txt',
 'rihanna.txt',
 'bjork.txt']

In [5]:
# Armar el dataset utilizando salto de línea para separar las oraciones/docs
df = pd.read_csv('songs_dataset/michael-jackson.txt', sep='/n', header=None)
df.head()

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


,0
0,[Spoken Intro:]
1,You ever want something
2,that you know you shouldn't have
3,"The more you know you shouldn't have it,"
4,The more you want it


In [6]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 9484


### 1 - Preprocesamiento

In [7]:
from keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

In [8]:
# Demos un vistazo
sentence_tokens[:2]

[['spoken', 'intro'], ['you', 'ever', 'want', 'something']]

### 2 - Crear los vectores (word2vec)

In [9]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobracargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [10]:
# Crearmos el modelo generador de vectoeres
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,       # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,          # cant de palabras antes y desp de la predicha
                     size=300,          # dimensionalidad de los vectores 
                     negative=20,       # cantidad de negative samples... 0 es no se usa
                     workers=1,         # si tienen más cores pueden cambiar este valor
                     sg=1)              # modelo 0:CBOW  1:skipgram

In [11]:
# Buildear el vocabularui con los tokens
w2v_model.build_vocab(sentence_tokens)

In [12]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 9484


In [13]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.vocab))

Cantidad de words distintas en el corpus: 1000


### 3 - Entrenar el modelo generador

In [14]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=20,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 338319.84375
Loss after epoch 1: 227767.71875
Loss after epoch 2: 228929.4375
Loss after epoch 3: 225359.4375
Loss after epoch 4: 196104.4375
Loss after epoch 5: 188187.375
Loss after epoch 6: 181549.75
Loss after epoch 7: 174775.0
Loss after epoch 8: 171784.75
Loss after epoch 9: 165706.75
Loss after epoch 10: 146940.75
Loss after epoch 11: 145552.0
Loss after epoch 12: 143006.0
Loss after epoch 13: 141207.75
Loss after epoch 14: 140830.5
Loss after epoch 15: 138899.25
Loss after epoch 16: 139130.5
Loss after epoch 17: 137745.0
Loss after epoch 18: 138696.75
Loss after epoch 19: 137433.0


(626341, 993780)

### 4 - Ensayar

In [15]:
# Palabras que MÁS se relacionan con...:
print("Palabras que MÁS se relacionan con...:")
print("* thriller \n", w2v_model.wv.most_similar(positive=["thriller"], topn=10))
print("* remember \n", w2v_model.wv.most_similar(positive=["remember"], topn=10))
print("* bad \n", w2v_model.wv.most_similar(positive=["bad"], topn=10))

Palabras que MÁS se relacionan con...:
* thriller 
 [('winds', 0.8437806963920593), ('till', 0.8355855345726013), ('killer', 0.8253421783447266), ('big', 0.8039060831069946), ('darkness', 0.802476167678833), ('s', 0.7975853085517883), ('breaking', 0.7926170825958252), ('injustice', 0.7789288759231567), ('history', 0.7749476432800293), ('lovers', 0.774057149887085)]
* remember 
 [('doot', 0.7765475511550903), ('after', 0.7477302551269531), ('great', 0.7366161346435547), ('means', 0.7320057153701782), ('times', 0.7310488224029541), ('perfect', 0.7270776033401489), ('thrill', 0.7086794376373291), ('play', 0.6900840997695923), ('choice', 0.689246654510498), ('end', 0.6818466186523438)]
* bad 
 [('shameone', 0.9027432799339294), ('smooth', 0.8830131888389587), ('laughing', 0.816936731338501), ('fast', 0.8148630261421204), ('bothered', 0.7877052426338196), ('telling', 0.7783938646316528), ('easy', 0.7735446691513062), ('late', 0.7663449645042419), ('everyday', 0.76183021068573), ('talking', 

In [16]:
# Palabras que MENOS se relacionan con...:
print("Palabras que MENOS se relacionan con...:")
print("* thriller \n", w2v_model.wv.most_similar(negative=["thriller"], topn=10))
print("* remember \n", w2v_model.wv.most_similar(negative=["remember"], topn=10))
print("* bad \n", w2v_model.wv.most_similar(negative=["bad"], topn=10))

Palabras que MENOS se relacionan con...:
* thriller 
 [('uuh', 0.10693293064832687), ('stevie', 0.09019403904676437), ('pitbull', 0.021125249564647675), ('siedah', 0.01513645052909851), ('cheater', -0.02849094569683075), ('happens', -0.10146676003932953), ('let', -0.11573702841997147), ('whatever', -0.11907555907964706), ("don't", -0.13370513916015625), ('refrain', -0.1360141634941101)]
* remember 
 [('stevie', 0.05462115257978439), ('uuh', 0.0044644735753536224), ('pitbull', -0.014146354049444199), ('cheater', -0.02320026606321335), ('siedah', -0.03485107421875), ('refrain', -0.06804600358009338), ('down', -0.09345267713069916), ('high', -0.10298650711774826), ('your', -0.12489107251167297), ('enough', -0.1285310685634613)]
* bad 
 [('pitbull', 0.07984061539173126), ('uuh', 0.06686697900295258), ('stevie', -0.004616007208824158), ('siedah', -0.006680287420749664), ('refrain', -0.07219935953617096), ('change', -0.08841358125209808), ('cheater', -0.09293422102928162), ('blow', -0.099818

In [17]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["billie"], topn=10)

[('jean', 0.9878240823745728),
 ('son', 0.9032402038574219),
 ('lover', 0.8783435821533203),
 ('kid', 0.8709703683853149),
 ('sleep', 0.842766284942627),
 ('dangerous', 0.825167179107666),
 ('yours', 0.8223621845245361),
 ('s', 0.8167554140090942),
 ('dom', 0.8162959814071655),
 ('sheldon', 0.8128227591514587)]

In [18]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["dance"], topn=5)

[('floor', 0.8353102803230286),
 ('across', 0.8314943313598633),
 ('carpet', 0.7830660343170166),
 ('boogie', 0.7821351885795593),
 ('earth', 0.7762736082077026)]

In [19]:
# Ensayar con una palabra que no está en el corpus (en vocab):
w2v_model.wv.most_similar(negative=["diedaa"])

KeyError: ignored

### 5 - Visualizar agrupación de vectores

In [20]:
from sklearn.decomposition import IncrementalPCA    
from sklearn.manifold import TSNE                   
import numpy as np                                  

def reduce_dimensions(model):
    num_dimensions = 2  

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index2word)  

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
    return x_vals, y_vals, labels

In [21]:
!pip install plotly nbformat==4.2.0

     |████████████████████████████████| 153 kB 7.6 MB/s 
  Attempting uninstall: nbformat
    Found existing installation: nbformat 5.2.0
    Uninstalling nbformat-5.2.0:
      Successfully uninstalled nbformat-5.2.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
nbconvert 5.6.1 requires nbformat>=4.4, but you have nbformat 4.2.0 which is incompatible.
nbclient 0.5.13 requires jupyter-client>=6.1.5, but you have jupyter-client 5.3.5 which is incompatible.
nbclient 0.5.13 requires nbformat>=5.0, but you have nbformat 4.2.0 which is incompatible.


In [22]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

x_vals, y_vals, labels = reduce_dimensions(w2v_model)

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:783: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:793: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  FutureWarning,


In [23]:
MAX_WORDS=200
fig = px.scatter(x=x_vals[:MAX_WORDS], y=y_vals[:MAX_WORDS], text=labels[:MAX_WORDS])
fig.show() # esto para plotly en colab